# General dataset 3

I need this to work on multiple cell types, idea is it can load in the values for diffeerrent cell types based on the index

In [1]:
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.dataloaders.datasets.general_dataset import GeneralDataset
dataset = GeneralDataset(
    split='train',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/test_chrom_dnase_chunkchrom.zarr',
    length=524288,
    mlm=0.25,
    acc_mlm=0.25,
    data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune.json'
)

replacing cell type number with data indices


In [2]:
#let's test an output
dataset.sequences

,0,1,2,3
0,chr1,0,524288,train
1,chr1,262144,786432,train
2,chr1,524288,1048576,train
3,chr1,786432,1310720,train
4,chr1,1048576,1572864,train
...,...,...,...,...
8879,chrX,154402816,154927104,train
8880,chrX,154664960,155189248,train
8881,chrX,154927104,155451392,train
8882,chrX,155189248,155713536,train


In [3]:
out1 = dataset[0]
out1[0][1][:,:10]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
import numpy as np
np.allclose(out1[0][1], dataset.data['chr1'][0,0:524288]) #not cell ty pe 0, want the first one of the list we defined

False

In [6]:
celltype_idx = dataset.data_idxs[0]
np.allclose(out1[0][1], dataset.data['chr1'][celltype_idx,0:524288])

False

In [7]:
out1[0][1].shape, dataset.data['chr1'][celltype_idx,0:524288].shape

(torch.Size([2, 524288]), (524288,))

In [9]:
np.allclose(out1[1][1][:,0], dataset.data['chr1'][celltype_idx,0:524288])

True

In [10]:
#let's choose 5 indices at random between 0 and 8883
import random
rand_idxs = random.sample(range(0, 8883), 5)
rand_idxs

[2277, 4262, 1087, 8842, 8870]

In [13]:
for idx in rand_idxs:
    out = dataset[idx]
    chrom, start, end, split = dataset.sequences.iloc[idx]
    print(np.allclose(out[1][1][:,0], dataset.data[chrom][celltype_idx,start:end]))

True
True
True
True
True


In [ ]:
dataset[0][1][0][:,0].shape

torch.Size([524288])

In [21]:
#works for celltype 0, now let's see if it makes seure all sequences are the same
seqs = []
i = rand_idxs[-1]
while i < len(dataset):
    seqs.append(dataset[i][1][0][:,0]) #seocnd one is the mask, that might differ, seq should be identical
    # print(seqs[0].shape)
    # break
    i = i + len(dataset.sequences)
    print('done with idx', i)
for i in range(len(seqs)-1):
    print(np.allclose(seqs[i], seqs[i+1]))

done with idx 17754
done with idx 26638
done with idx 35522
done with idx 44406
done with idx 53290
done with idx 62174
done with idx 71058
True
True
True
True
True
True


In [ ]:
print(len(seqs))
print(seqs[0][233333:233333+30])
print(seqs[1][233333:233333+30]) #why just A and C?

7
tensor([0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.])
tensor([0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.])


In [ ]:
s = 0
print(seqs[0][s:s+30])
print(seqs[1][s:s+30])
#something is almost definitely wrong??

tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])


In [26]:
out = dataset[i]
out[1][0].shape

torch.Size([524288, 6])

In [27]:
#oh, because it's one hot encoded lmfao!!!!

seqs = []
i = rand_idxs[-1]
while i < len(dataset):
    seqs.append(dataset[i][1][0][:,:-1]) #last is the mask!
    # print(seqs[0].shape)
    # break
    i = i + len(dataset.sequences)
    print('done with idx', i)
for i in range(len(seqs)-1):
    print(np.allclose(seqs[i], seqs[i+1]))

done with idx 17754
done with idx 26638
done with idx 35522
done with idx 44406
done with idx 53290
done with idx 62174
done with idx 71058
True
True
True
True
True
True


In [ ]:
print(len(seqs))
print(seqs[0][233333:233333+10])
print(seqs[1][233333:233333+10])
#this is more like what you would expect!!

7
tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])
tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])


In [30]:
#now let's find some truly random regions!

rand_idxs = random.sample(range(0, len(dataset)), 15)
print(rand_idxs)
for idx in rand_idxs:
    out = dataset[idx]
    celltype_idx = dataset.data_idxs[idx//len(dataset.sequences)] #takes into account
    seq_idx = idx % len(dataset.sequences)
    chrom, start, end, split = dataset.sequences.iloc[seq_idx]
    print(np.allclose(out[1][1][:,0], dataset.data[chrom][celltype_idx,start:end]))

[47493, 57010, 38327, 46623, 30170, 24573, 47601, 4079, 16506, 18139, 22511, 7821, 8241, 61689, 58275]
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
#so it does as we think, finding the right sequence and cell type and then simply grabs those values and then does masking but we looking at true values here!

# now testing it

Added a lot of features like load expression data and can multitask or find the right index

In [ ]:
import json
import torch
import numpy as np
import pandas as pd
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.dataloaders.datasets.general_dataset import GeneralDataset

#to finetune on Enformer CAGE
dataset = GeneralDataset(
    split='train',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/dnase_chunkchrom_processed.zarr',
    sequences_bed_file='/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed',
    length=524288,
    mlm=0,
    acc_mlm=0,
    additional_data='/data1/lesliec/sarthak/data/enformer/data/labels.zarr',
    additional_data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/nob_immune_CAGE.json',
    data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/nob_immune.json',
)

#tells it to get train split, then uses base pair DNase for input, then predicts the pooled values from the Enformer CAGE data
#data_idxs tells it to subset to thoe indices

print(dataset.additional_data_idxs) #CAGE values
print(dataset.data_idxs) #DNase values

replacing cell type number with data indices
[4759 4760 4831 4764 4758]
[457 177 120 522 392]


In [ ]:
print(len(dataset))
print(len(dataset.sequences))
print(len(dataset.additional_data_idxs))
print(len(dataset.additional_data_idxs)* len(dataset.sequences)) #it's cell types times sequences

170105
34021
5
170105


In [7]:
#so let's get a random index between 0 and 170105
import random
rand_idx = random.sample(range(0, 170105), 1)[0]
print(rand_idx)

40003


In [12]:
out = dataset[rand_idx]
print(out[0][0].shape, out[0][1].shape, out[1][0].shape, out[1][1].shape, out[1][2].shape)

torch.Size([6, 524288]) torch.Size([2, 524288]) torch.Size([524288, 6]) torch.Size([524288, 2]) (896, 1)


In [10]:
#so based on our approach we would do this. First get cell type information by doing //34021 th eoriginal amount of sequences

celltype_idx = rand_idx // 34021
print(celltype_idx)

#then get the sequence info by doing % 34021
seq_idx = rand_idx % 34021
print(seq_idx)

#also it won't be that actual index sicne we're using a subset of the indices. Just logically looking it will be index 177 for Dnase and 4760 for CAGE

1
5982


In [11]:
dataset.sequences.iloc[seq_idx] #this is the sequence info

0        chr11
1    133564200
2    133695272
3        train
Name: 5982, dtype: object

In [13]:
#let's grab chrom, start, end, split
chrom, start, end, split = dataset.sequences.iloc[seq_idx]
#now have to adjust start and end to be length 524288
length = end - start
if length < 524288:
    start = start - (524288 - length) // 2
    end = end + (524288 - length) // 2

print(chrom, start, end, split, end-start)

chr11 133367592 133891880 train 524288


In [14]:
#now let's grab these values
seq = dataset.genome[chrom][start:end]
seq.shape

(524288,)

In [15]:
seq

array([ 7,  9, 10, ...,  7,  8,  8], dtype=int8)

In [ ]:
out[0][0] #seems to allign, but let's check

tensor([[1., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [17]:
11 in seq

False

In [24]:
torch.tensor(seq-7, dtype=torch.int64).shape

torch.Size([524288])

In [25]:
#so let's do torch.nn.onehot
seq_onehot = torch.nn.functional.one_hot(torch.tensor(seq-7, dtype=torch.int64), num_classes=4)
seq_onehot.shape

torch.Size([524288, 4])

In [26]:
out[0][0].shape

torch.Size([6, 524288])

In [29]:
torch.allclose(out[0][0][:4].t().to(torch.int64), seq_onehot) #so this is the one hot encoded version of the sequence

True

In [ ]:
#sequence is the easy part, the hard part is DNase then CAGE
dataset.data[chrom].shape

(674, 135086622)

In [ ]:
data = dataset.data[chrom][celltype_idx, start:end]
data.shape

(524288,)

In [33]:
#now let's see if this matches?
torch.allclose(out[0][1][0], torch.tensor(data, dtype=torch.float32)) #so this is the DNase data

False

In [34]:
#hmmm, let's see
#ahh celltype idx is 1, we want the 177 or whatever
true_celltype_idx = dataset.data_idxs[celltype_idx]
print(true_celltype_idx)

177


In [35]:
data = dataset.data[chrom][true_celltype_idx, start:end]
torch.allclose(out[0][1][0], torch.tensor(data, dtype=torch.float32)) #so this is the DNase data

True

In [37]:
#perfect! This all matches as we expect!!
#final thing is CAGE, not too bad, similar concept
true_celltype_idx = dataset.additional_data_idxs[celltype_idx]
print(true_celltype_idx)
extra_data = dataset.additional_data[split][seq_idx]
print(extra_data.shape)

4760
(896, 5313)


In [38]:
extra_data = extra_data[:,true_celltype_idx]
print(extra_data.shape)

(896,)


In [41]:
out[1][2].shape

(896, 1)

In [ ]:
#now let's compare
np.allclose(out[1][2][:,0], extra_data) #so this is the CAGE data
#this matches as well, because the key is to use the sequence index!!

True

In [44]:
#I'm now quite confident this is good. Just some housekeeping
np.allclose(out[0][0][:5], out[1][0][:,:5].T) #so this is the one hot encoded version of the sequence

True

In [ ]:
out[0][0][5].max(), out[0][0][5].min() #all zeros for the mask column!

(tensor(0.), tensor(0.))

In [48]:
out[0][1].shape

torch.Size([2, 524288])

In [ ]:
np.allclose(out[0][1][0,:], out[1][1][:,0]) #so this is the DNase data, all data matches

True

In [ ]:
out[0][1][1].max(), out[0][1][1].min() #all zeros for the mask column!

(tensor(0.), tensor(0.))

In [ ]:
#so the data looks good then!!! I am happy with the dataset!

# how to train for RNA seq

In [ ]:
#the idea is we have models training to predict RNA seq
#but we are only inputting DNAse seq for one?

#wait yeah it's fine because accessibility input of DNase and then predict all DM12878 tracks, that's fine, only need the one input!

# training on enformer task

In [ ]:
'''
├── dataset
│   └── _name_: GeneralLoader                                                                                                                                                                                                                                                                                                                                                           
│       dataset_name: GeneralLoader                                                                                                                                                                                                                                                                                                                                                     
│       batch_size_eval: 2                                                                                                                                                                                                                                                                                                                                                              
│       shuffle: true                                                                                                                                                                                                                                                                                                                                                                   
│       pin_memory: true                                                                                                                                                                                                                                                                                                                                                                
│       batch_size: 1                                                                                                                                                                                                                                                                                                                                                                   
│       num_workers: 1                                                                                                                                                                                                                                                                                                                                                                  
│       data_path: /data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/cell_type_arrays/GM12878_DNase.npz                                                                                                                                                                                                                                                                                   
│       length: 524288                                                                                                                                                                                                                                                                                                                                                                  
│       shift_sequences: 0                                                                                                                                                                                                                                                                                                                                                              
│       load_in: false                                                                                                                                                                                                                                                                                                                                                                  
│       rc_aug: false                                                                                                                                                                                                                                                                                                                                                                   
│       mlm: 0                                                                                                                                                                                                                                                                                                                                                                          
│       acc_mlm: 1                                                                                                                                                                                                                                                                                                                                                                      
│       acc_type: continuous                                                                                                                                                                                                                                                                                                                                                            
│       acc_mask_size: 500                                                                                                                                                                                                                                                                                                                                                              
│       weight_peaks: false                                                                                                                                                                                                                                                                                                                                                             
│       additional_data: /data1/lesliec/sarthak/data/enformer/data/labels.zarr                                                                                                                                                                                                                                                                                                          
│       sequences_bed_file: /data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed  

'''

#these are the options, let's do it
import json
import torch
import numpy as np
import pandas as pd
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.dataloaders.datasets.general_dataset import GeneralDataset

dataset = GeneralDataset(
    split='train',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/cell_type_arrays/GM12878_DNase.npz',
    sequences_bed_file='/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed',
    length=524288,
    shift_sequences=0,
    mlm=0,
    acc_mlm=1,
    additional_data='/data1/lesliec/sarthak/data/enformer/data/labels.zarr',
)


NameError: name 'datset' is not defined

In [2]:
out = dataset[0]
out[0][0].shape, out[0][1].shape, out[1][0].shape, out[1][1].shape, out[1][2].shape

(torch.Size([6, 524288]),
 torch.Size([2, 524288]),
 torch.Size([524288, 6]),
 torch.Size([524288, 2]),
 (896, 5313))

In [ ]:
#looks like the right sized output!
out[0][1].sum() #wait, there should be no data...

tensor(427990.1875)

In [4]:
out[0][1]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [5]:
out[0][1].sum(1)

tensor([  8702.2129, 419288.0000])

In [6]:
#let's do mask only!

dataset = GeneralDataset(
    split='train',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/cell_type_arrays/GM12878_DNase.npz',
    sequences_bed_file='/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed',
    length=524288,
    shift_sequences=0,
    mlm=0,
    acc_mlm=1,
    additional_data='/data1/lesliec/sarthak/data/enformer/data/labels.zarr',
    mask_only=True,
)

out = dataset[0]
out[0][0].shape, out[0][1].shape, out[1][0].shape, out[1][1].shape, out[1][2].shape

(torch.Size([6, 524288]),
 torch.Size([2, 524288]),
 torch.Size([524288, 6]),
 torch.Size([524288, 2]),
 (896, 5313))

In [ ]:
out[0][1].sum() #yup, just the mask!

tensor(524288.)

In [9]:
out[0][1].sum(1)

tensor([     0., 524288.])

In [ ]:
#let's look at the other data
out[1][2] #the coveratge, I believe this is fine? Uses t index, so it's all good!!

array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

# verifying the fine tuning of multiple cell type models

In [1]:
#we need to both load the proper DNase and exxpression data. Let's make sure it works and that it makes sense
#let's see what the config says

'''
dataset:
  _name_: GeneralLoader
  dataset_name: GeneralLoader
  batch_size_eval: ${eval:${.batch_size} * 2}
  shuffle: true
  pin_memory: true
  batch_size: 1
  num_workers: 7
  data_path: /data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/dnase_chunkchrom_processed.zarr
  length: 524288
  shift_sequences: 0
  load_in: false
  rc_aug: false
  mlm: 0
  acc_mlm: 0
  acc_type: continuous
  acc_mask_size: 500
  weight_peaks: false
  additional_data: /data1/lesliec/sarthak/data/enformer/data/labels.zarr
  sequences_bed_file: /data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed
  additional_data_idxs: /data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune_CAGE.json
  data_idxs: /data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune.json

'''

#I assume it's correct since the benchmarks seem quite decent?

import torch
import numpy as np
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.dataloaders.datasets.general_dataset import GeneralDataset

dataset = GeneralDataset(
    split='train',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/dnase_chunkchrom_processed.zarr',
    sequences_bed_file='/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed',
    length=524288,
    shift_sequences=0,
    mlm=0,
    acc_mlm=0,
    additional_data='/data1/lesliec/sarthak/data/enformer/data/labels.zarr',
    additional_data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune_CAGE.json',
    data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune.json',
)

replacing cell type number with data indices, 7


In [2]:
dataset.additional_data_idxs

array([4759, 4760, 5117, 4831, 4764, 4758, 5110])

In [3]:
dataset.data_idxs

array([457, 177, 354, 120, 522, 392,  12])

In [4]:
#let's just manually verify some things
import pandas as pd
targets_file = '/data1/lesliec/sarthak/data/enformer/data/targets.txt'
df = pd.read_csv(targets_file,sep="\t")
df

,index,genome,identifier,file,clip,scale,sum_stat,description
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion
3,3,0,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...
4,4,0,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348
...,...,...,...,...,...,...,...,...
5308,5308,0,CNhs14239,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:epithelioid sarcoma cell line:HS-ES-2R
5309,5309,0,CNhs14240,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:squamous cell lung carcinoma cell line:RE...
5310,5310,0,CNhs14241,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:gastric cancer cell line:GSS
5311,5311,0,CNhs14244,/home/drk/tillage/datasets/human/cage/fantom/C...,384,1,sum,CAGE:carcinoid cell line:NCI-H727


In [5]:
for i in range(dataset.data_idxs.shape[0]):
    print(df.iloc[dataset.data_idxs[i]]['description'])
    print(df.iloc[dataset.additional_data_idxs[i]]['description'])
    print('---')

DNASE:CD4-positive, alpha-beta T cell male adult (21 year)
CAGE:CD4+ T Cells,
---
DNASE:CD8-positive, alpha-beta T cell male adult (21 year)
CAGE:CD8+ T Cells,
---
DNASE:B cell male adult (21 year)
CAGE:CD19+ B Cells,
---
DNASE:Jurkat clone E61
CAGE:acute lymphoblastic leukemia (T-ALL) cell line:Jurkat
---
DNASE:natural killer cell male adult (21 year)
CAGE:Natural Killer Cells,
---
DNASE:CD14-positive monocyte male adult (21 year)
CAGE:CD14+ Monocytes,
---
DNASE:GM12878
CAGE:B lymphoblastoid cell line: GM12878 ENCODE, biol_
---


In [8]:
#seems like it is correct in how they're paired, now let's see an example
sequences_bed_file = '/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed'
sequences_bed = pd.read_csv(sequences_bed_file, sep="\t", header=None)
sequences_bed = sequences_bed[sequences_bed[3] == 'train']
sequences_bed

,0,1,2,3
0,chr18,928386,1059458,train
1,chr4,113630947,113762019,train
2,chr11,18427720,18558792,train
3,chr16,85805681,85936753,train
4,chr3,158386188,158517260,train
...,...,...,...,...
34016,chr7,50515122,50646194,train
34017,chr7,135602769,135733841,train
34018,chr4,189004198,189135270,train
34019,chr4,10438099,10569171,train


In [9]:
len(dataset), 34021*7

(238147, 238147)

In [10]:
#let's look at sequence 1
out = dataset[1]
out[0][0].shape, out[0][1].shape, out[1][0].shape, out[1][1].shape, out[1][2].shape

(torch.Size([6, 524288]),
 torch.Size([2, 524288]),
 torch.Size([524288, 6]),
 torch.Size([524288, 2]),
 (896, 1))

In [11]:
chrom = sequences_bed.iloc[1][0]
start = sequences_bed.iloc[1][1]
end = sequences_bed.iloc[1][2]
end-start

131072

In [12]:
#have to extend it to 524288
length = end - start
if length < 524288:
    start = start - (524288 - length) // 2
    end = end + (524288 - length) // 2
print(chrom, start, end)

chr4 113434339 113958627


In [13]:
seq = dataset.genome[chrom][start:end]
seq.shape

(524288,)

In [16]:
seq

array([7, 7, 9, ..., 8, 9, 7], dtype=int8)

In [22]:
torch.tensor(seq)

tensor([7, 7, 9,  ..., 8, 9, 7], dtype=torch.int8)

In [23]:
out[1][0].argmax(1)+7

tensor([7, 7, 9,  ..., 8, 9, 7])

In [25]:
torch.allclose(out[1][0].argmax(1).to(torch.int8)+7, torch.tensor(seq))

True

In [26]:
#so the sequence seems right, let's make sure we get the right data for accessibility
acc = dataset.data[chrom][dataset.data_idxs[0], start:end]
acc.shape

(524288,)

In [27]:
out[1][1].shape

torch.Size([524288, 2])

In [ ]:
torch.allclose(out[1][1][:,0], torch.tensor(acc, dtype=torch.float32)) #so we are loading the right accessibility data

True

In [31]:
#now let's do the expression from CAGE
dataset.additional_data.tree()

/
├── test (1937, 896, 5313) float32
├── train (34021, 896, 5313) float32
└── val (2213, 896, 5313) float32

In [33]:
exp = dataset.additional_data['train'][1]

In [36]:
#now we have to get the right sub value
exp_data = exp[:,dataset.additional_data_idxs[0]]
np.allclose(out[1][2][:,0], exp_data) #so this is the CAGE data

True

In [37]:
#ok so we loaded the right expression data as well

#now to verify, let's look at a different cell type
out = dataset[1+34021]

In [ ]:
torch.allclose(out[1][0].argmax(1).to(torch.int8)+7, torch.tensor(seq)) #so sequence is the same

True

In [40]:
#need to get a new accessibility value
acc = dataset.data[chrom][dataset.data_idxs[1], start:end]
torch.allclose(out[1][1][:,0], torch.tensor(acc, dtype=torch.float32)) #so we are loading the right accessibility data

True

In [41]:
#and finally expression
exp_data = exp[:,dataset.additional_data_idxs[1]]
np.allclose(out[1][2][:,0], exp_data) #so this is the CAGE data

True

In [42]:
#let's make a loop that chooses 10 random indices and then checks all the celltypes

random_idxs = np.random.randint(0, 34021, 10)
random_idxs

array([23675, 16069, 31029, 14471, 30114,  2220,  5239,  4780, 18921,
       21299])

In [46]:
for idx in random_idxs:
    
    chrom = sequences_bed.iloc[idx][0]
    start = sequences_bed.iloc[idx][1]
    end = sequences_bed.iloc[idx][2]
    length = end - start
    if length < 524288:
        start = start - (524288 - length) // 2
        end = end + (524288 - length) // 2
    # print(chrom, start, end)
    
    seq = dataset.genome[chrom][start:end]
    
    exp = dataset.additional_data['train'][idx]
    
    for i in range(7):
        out = dataset[idx + i*34021]
        #first assert the sequence
        assert torch.allclose(out[1][0].argmax(1).to(torch.int8)+7, torch.tensor(seq))
        #then assert the accessibility
        acc = dataset.data[chrom][dataset.data_idxs[i], start:end]
        assert torch.allclose(out[1][1][:,0], torch.tensor(acc, dtype=torch.float32))
        #finally assert the expression
        exp_data = exp[:,dataset.additional_data_idxs[i]]
        assert np.allclose(out[1][2][:,0], exp_data)
        print('done with idx', idx + i*34021)
    
    # print('done with idx', idx)

done with idx 23675
done with idx 57696
done with idx 91717
done with idx 125738
done with idx 159759
done with idx 193780
done with idx 227801
done with idx 16069
done with idx 50090
done with idx 84111
done with idx 118132
done with idx 152153
done with idx 186174
done with idx 220195
done with idx 31029
done with idx 65050
done with idx 99071
done with idx 133092
done with idx 167113
done with idx 201134
done with idx 235155
done with idx 14471
done with idx 48492
done with idx 82513
done with idx 116534
done with idx 150555
done with idx 184576
done with idx 218597
done with idx 30114
done with idx 64135
done with idx 98156
done with idx 132177
done with idx 166198
done with idx 200219
done with idx 234240
done with idx 2220
done with idx 36241
done with idx 70262
done with idx 104283
done with idx 138304
done with idx 172325
done with idx 206346
done with idx 5239
done with idx 39260
done with idx 73281
done with idx 107302
done with idx 141323
done with idx 175344
done with idx 2

In [47]:
#and let's just quickly check this with the validation set too

dataset = GeneralDataset(
    split='val',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/dnase_chunkchrom_processed.zarr',
    sequences_bed_file='/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed',
    length=524288,
    shift_sequences=0,
    mlm=0,
    acc_mlm=0,
    additional_data='/data1/lesliec/sarthak/data/enformer/data/labels.zarr',
    additional_data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune_CAGE.json',
    data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune.json',
)

replacing cell type number with data indices, 7


In [50]:
sequences_bed_file = '/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed'
sequences_bed = pd.read_csv(sequences_bed_file, sep="\t", header=None)
sequences_bed = sequences_bed[sequences_bed[3] == 'valid']
sequences_bed

,0,1,2,3
34021,chr6,165740202,165871274,valid
34022,chrX,55044496,55175568,valid
34023,chrX,84489673,84620745,valid
34024,chrX,26382093,26513165,valid
34025,chr7,2304644,2435716,valid
...,...,...,...,...
36229,chrX,16977595,17108667,valid
36230,chr20,45038994,45170066,valid
36231,chrX,24547069,24678141,valid
36232,chr2,235793611,235924683,valid


In [54]:
random_idxs = np.random.randint(0, len(sequences_bed), 10)
random_idxs

array([1391, 1316, 1715,  212, 2207, 1777, 2046,  914, 2062, 1146])

In [ ]:
#so have to adjust it not to be 34021, but whatever the validation set length is
for idx in random_idxs:
    
    chrom = sequences_bed.iloc[idx][0]
    start = sequences_bed.iloc[idx][1]
    end = sequences_bed.iloc[idx][2]
    length = end - start
    if length < 524288:
        start = start - (524288 - length) // 2
        end = end + (524288 - length) // 2
    # print(chrom, start, end)
    
    seq = dataset.genome[chrom][start:end]
    
    exp = dataset.additional_data['val'][idx]
    
    for i in range(7):
        out = dataset[idx + i*len(sequences_bed)]
        #first assert the sequence
        assert torch.allclose(out[1][0].argmax(1).to(torch.int8)+7, torch.tensor(seq))
        #then assert the accessibility
        acc = dataset.data[chrom][dataset.data_idxs[i], start:end]
        assert torch.allclose(out[1][1][:,0], torch.tensor(acc, dtype=torch.float32))
        #finally assert the expression
        exp_data = exp[:,dataset.additional_data_idxs[i]]
        assert np.allclose(out[1][2][:,0], exp_data)
        print('done with idx', idx + i*len(sequences_bed))

done with idx 1391
done with idx 3604
done with idx 5817
done with idx 8030
done with idx 10243
done with idx 12456
done with idx 14669
done with idx 1316
done with idx 3529
done with idx 5742
done with idx 7955
done with idx 10168
done with idx 12381
done with idx 14594
done with idx 1715
done with idx 3928
done with idx 6141
done with idx 8354
done with idx 10567
done with idx 12780
done with idx 14993
done with idx 212
done with idx 2425
done with idx 4638
done with idx 6851
done with idx 9064
done with idx 11277
done with idx 13490
done with idx 2207
done with idx 4420
done with idx 6633
done with idx 8846
done with idx 11059
done with idx 13272
done with idx 15485
done with idx 1777
done with idx 3990
done with idx 6203
done with idx 8416
done with idx 10629
done with idx 12842
done with idx 15055
done with idx 2046
done with idx 4259
done with idx 6472
done with idx 8685
done with idx 10898
done with idx 13111
done with idx 15324
done with idx 914
done with idx 3127
done with idx

In [ ]:
#To be clear tho, this only works because the enformer targets starts with DNase. It means that we can simply input enformer index into it when normally we'd have to change it.
#we can use the targets dnase too? But here it's the same lol
#regardless, it shows what we're loading in is working, so validation loss being so strange just shows it's very tough to actually do this task...